In [29]:
import sys
sys.path.append('..')
import gluonbook as gb
import mxnet as mx
from mxnet import nd, gluon, init
from mxnet.gluon import nn

def vgg_block(num_convs, num_channels):
    blk = nn.Sequential()
    for _ in range(num_convs):
        blk.add(nn.Conv2D(channels=num_channels,
                          kernel_size=3, 
                          padding=1, 
                          activation='relu'))
    blk.add(nn.MaxPool2D(pool_size=2))
    return blk

def vgg(conv_arch, num_outputs):
    net = nn.Sequential()
    for (num_convs, num_channels) in conv_arch:
        net.add(vgg_block(num_convs, num_channels))
    net.add(
            nn.Dense(512, activation='relu'),
            nn.Dropout(0.5),
            nn.Dense(512, activation='relu'),
            nn.Dropout(0.5),
            nn.Dense(num_outputs),
            )
    return net


In [51]:
conv_arch = ((1,64), (1,128), (2,256), (2,512), (2,512))
net = vgg(conv_arch, 1000)
net.initialize()

X = nd.random.uniform(shape=(33,3,224,224))

for blk in net:
    X = blk(X)
    print(blk.name, ' shape:\t', X.shape)

sequential80  shape:	 (33, 64, 112, 112)
sequential81  shape:	 (33, 128, 56, 56)
sequential82  shape:	 (33, 256, 28, 28)
sequential83  shape:	 (33, 512, 14, 14)
sequential84  shape:	 (33, 512, 7, 7)
dense42  shape:	 (33, 512)
dropout26  shape:	 (33, 512)
dense43  shape:	 (33, 512)
dropout27  shape:	 (33, 512)
dense44  shape:	 (33, 1000)


In [52]:
ratio =  2
small_conv_arch = [(pair[0], int(pair[1]/ratio)) for pair in conv_arch]
net = vgg(small_conv_arch, 10)

In [ ]:
ctx = gb.try_gpu()
net.initialize(ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.05})

train_data, test_data = gb.load_data_fashion_mnist(batch_size=128, resize=224)
loss = gluon.loss.SoftmaxCrossEntropyLoss()
gb.train(train_data, test_data, net, loss, trainer, ctx, num_epochs=3)

/Users/jiananwei/anaconda3/envs/ML/lib/python3.6/site-packages/mxnet/gluon/data/vision/datasets.py:118: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  label = np.fromstring(fin.read(), dtype=np.uint8).astype(np.int32)
/Users/jiananwei/anaconda3/envs/ML/lib/python3.6/site-packages/mxnet/gluon/data/vision/datasets.py:122: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data = np.fromstring(fin.read(), dtype=np.uint8)


training on  cpu(0)
